# Introduction

### Imports

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

# Options for pandas
pd.options.display.max_columns = 150
pd.options.display.max_rows = 150

/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Imports</a></span></li></ul></li></ul></li></ul></div>

In [2]:
import numpy as np
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy import Spider
import scrapy
from scrapy import Request
import time
from collections import defaultdict

class usb_hi_Spider(scrapy.Spider):
        
    name = "surfboards"
    start_urls = ["https://www.usedsurfboardshawaii.com/?post_type=product"]
    
    def is_surfboard(self, response):
        
        surfboard_category_list = ['Big Guy Boards', 'Big Wave Gun', 'Boards from Pros', 'Bonzer', 'Carbon Fiber', 'Clearance',
         'Collector Boards', 'Epoxy', 'Fish', 'Five-fin', 'Foam ', 'Grom', 'Groveler', 'Gun', 'Hybrid',
         'Fun Board', 'Hybrid/Fun Board', 'Hydro Foil', 'Kite Boards', 'Longboard', 'Mini Simmons',
         'On Sale', 'Quad', 'Semi-gun', 'Shortboard', 'Single Fin', 'Step-up', 'SUP', 'Twin Fin',
         'Vintage Surfboards', 'Youth Shortboard']
        surfboard_category_list = [tag.lower() for tag in surfboard_category_list]
        
        categories = ' '.join(response.xpath('//*[@class="posted_in"]/a/text()').extract()).lower().split(' ')
        related_category = 0
        for category in categories:
            if category in surfboard_category_list:
                related_category += 1
        if related_category > 0:
            return True
    
    def attr_check_dict(self, response):
        attr_dict = defaultdict(lambda: None)
        for i in range(1,7):
            attr = ''.join(response.xpath('//*[@class="product_description"]/p[' + str(i) + ']/text()').extract())
            
            if attr == '':
                continue
                
            if re.search(r'future|fcs|glassed\s?on|lokbox|nsp|thruster|quad|single|twin|5[\s?\-]fin|5fin', attr.lower()) and i == 1:
                attr_dict['type'] = attr

            elif re.search('H(?=\s*\d+)', attr):
                attr_dict['height'] = attr

            elif re.search('W(?=\s*\d+)', attr):
                attr_dict['width'] = attr

            elif re.search('T(?=\s*\d+)', attr):
                attr_dict['thickness'] = attr

            elif re.search('V(?=\s*\d+)', attr):
                attr_dict['volume'] = attr

            elif i != 1:
                attr_dict['description'] = "".join(line for line in response.xpath('//*[@class="product_description"]/p[' + str(i) + ']/text()').extract())
                break
        
        
        sale_price = ''.join(response.xpath('//p[@class="price"]/ins/span[1]/text()').extract()).strip('$').replace(',','')
        regular_price = ''.join(response.xpath('//p[@class="price"]/span[1]/text()').extract()).strip('$').replace(',','') 
        
        if sale_price == '':
            attr_dict['price'] = float(regular_price)
        else:
            attr_dict['price'] = float(sale_price)
            
        attr_dict['url'] = response.meta.get('URL')
        attr_dict['title'] = ''.join(response.xpath('//h1[@class="product_title entry-title"]/text()').extract())
        attr_dict['condition'] = ''.join(response.xpath('//*[@class="woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_condition"]/td/p/text()').extract())
        attr_dict['manufacturer'] = ''.join(response.xpath('//*[@class="woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_shaper-of-board"]/td/p/text()').extract())
        
        return attr_dict

    def parse(self, response):
        products = response.xpath('//*[@id="products-grid"]/li')
        print('Total: ', len(products))
        for i,product in enumerate(products):
            print('Processing {} out of {}'.format(i+1, len(products)))
            
            relative_url = product.xpath('a/@href').extract_first()

            time.sleep(.1)

            yield Request(relative_url, callback=self.parse_page, meta={'URL': relative_url})

        relative_next_url = response.xpath('//*[@class="next page-numbers"]/@href').extract_first()
        
        print()
        print(relative_next_url)

        yield Request(relative_next_url, callback=self.parse)
        
    def parse_page(self, response):
        if self.is_surfboard(response) == True:
            attr_dict = self.attr_check_dict(response)
            yield {
            'url': attr_dict['url'],
            'title': attr_dict['title'],
            'price': attr_dict['price'],
            'description': attr_dict['description'],
            'type': attr_dict['type'],    
            'height': attr_dict['height'],
            'width': attr_dict['width'],    
            'thickness': attr_dict['thickness'],    
            'condition': attr_dict['condition'],
            'manufacturer': attr_dict['manufacturer']    
            }
process = CrawlerProcess({
'FEED_FORMAT': 'json',
'FEED_URI': 'used_surfboard_hawaii.json',
'ROBOTSTXT_OBEY': False,
'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
'AUTOTHROTTLE_ENABLED': True,
'HTTPCACHE_ENABLED': True,
'LOG_ENABLED': True
# 'DOWNLOAD_DELAY' : 1
})                             

process.crawl(usb_hi_Spider)
process.start()


2019-11-16 15:09:06 [scrapy.utils.log] INFO: Scrapy 1.7.3 started (bot: scrapybot)
2019-11-16 15:09:06 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.7.0, Python 3.7.1 (default, Dec 14 2018, 13:28:58) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 18.0.0 (OpenSSL 1.1.1a  20 Nov 2018), cryptography 2.4.2, Platform Darwin-18.7.0-x86_64-i386-64bit
2019-11-16 15:09:06 [scrapy.crawler] INFO: Overridden settings: {'AUTOTHROTTLE_ENABLED': True, 'FEED_FORMAT': 'json', 'FEED_URI': 'used_surfboard_hawaii.json', 'HTTPCACHE_ENABLED': True, 'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)'}
2019-11-16 15:09:06 [scrapy.extensions.telnet] INFO: Telnet Password: ee067c70f91f1be9
2019-11-16 15:09:06 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExpor

Total:  25
Processing 1 out of 25
Processing 2 out of 25


2019-11-16 15:09:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.usedsurfboardshawaii.com/product/7ft-4in-donald-takayama-scorpion-pin-tail-ref-30543/> (referer: https://www.usedsurfboardshawaii.com/?post_type=product) ['cached']
2019-11-16 15:09:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.usedsurfboardshawaii.com/product/9ft-peralta-round-pin-tail-ref-30460/> (referer: https://www.usedsurfboardshawaii.com/?post_type=product) ['cached']
2019-11-16 15:09:07 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.usedsurfboardshawaii.com/product/6ft-2in-tokoro-4vc-round-pin-tail-ref-30481/>
{'url': 'https://www.usedsurfboardshawaii.com/product/6ft-2in-tokoro-4vc-round-pin-tail-ref-30481/', 'title': '6ft 2in Tokoro (4VC) Round Pin Tail Ref# 30481', 'price': 585.0, 'description': 'Brand new shortboard with no signs of previous use', 'type': 'Thruster (FCS2)', 'height': 'H 6’2″', 'width': 'W 18 7/8″', 'thickness': 'T 2 3/8″', 'condition': 'Excellent, Good,

Processing 3 out of 25
Processing 4 out of 25


2019-11-16 15:09:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.usedsurfboardshawaii.com/product/9ft-donald-takayama-squash-tail-ref-30445/> (referer: https://www.usedsurfboardshawaii.com/?post_type=product) ['cached']
2019-11-16 15:09:07 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.usedsurfboardshawaii.com/product/7ft-4in-donald-takayama-scorpion-pin-tail-ref-30543/>
{'url': 'https://www.usedsurfboardshawaii.com/product/7ft-4in-donald-takayama-scorpion-pin-tail-ref-30543/', 'title': '7ft 4in Donald Takayama (Scorpion) Pin Tail Ref# 30543', 'price': 749.0, 'description': 'Excellent condition Hybrid Fun board with minimal signs of previous use', 'type': '4+1 (FCS)', 'height': 'H 7’4″', 'width': 'W 22″', 'thickness': 'T 2 1/2″', 'condition': 'Excellent, Good', 'manufacturer': 'Donald Takayama, Guy Takayama'}
2019-11-16 15:09:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.usedsurfboardshawaii.com/product/6ft-8in-eric-arakawa-mr-200-round-pin-tai

...

Processing 17 out of 20
Processing 18 out of 20


2019-11-16 15:11:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.usedsurfboardshawaii.com/product/used-surfboards-hawaii-shirts/> (referer: https://www.usedsurfboardshawaii.com/page/35/?post_type=product) ['cached']
2019-11-16 15:11:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.usedsurfboardshawaii.com/product/used-surfboards-hawaii-limited-edition-hat/> (referer: https://www.usedsurfboardshawaii.com/page/35/?post_type=product) ['cached']


Processing 19 out of 20
Processing 20 out of 20


2019-11-16 15:11:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.usedsurfboardshawaii.com/product/used-surfboards-hawaii-hat/> (referer: https://www.usedsurfboardshawaii.com/page/35/?post_type=product) ['cached']
2019-11-16 15:11:20 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.usedsurfboardshawaii.com/page/35/?post_type=product> (referer: https://www.usedsurfboardshawaii.com/page/34/?post_type=product)
Traceback (most recent call last):
  File "/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/scrapy/utils/defer.py", line 102, in iter_errback
    yield next(it)
  File "/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/scrapy/core/spidermw.py", line 84, in evaluate_iterable
    for r in iterable:
  File "/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/scrapy/spidermiddlewares/offsite.py", line 29, in process_spider_output
    for x in result:
  File "/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/scrapy/


None
